In [1]:
import pymongo
import re
from collections import OrderedDict
from datetime import datetime

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017", username="", password="")
raw_events_db = myclient["raw_events_db"]
events_collection = raw_events_db["events"]

applications_db = myclient['applications_db']
applications_collection= applications_db['applications']

In [3]:
applications_documents = applications_collection.find({},projection = {"_id":1,"name":1}).sort("name")

app_id_app_name_dict = {}

for x in applications_documents:
    app_id = x['_id'].strip()
    app_name = x['name'].strip()
    app_id_app_name_dict[app_id] = app_name
print("# Applications: ",len(app_id_app_name_dict))

# Applications:  419


In [4]:
#Get events per application
pipeline= [{"$group": {"_id": "$application_id", "count": {"$sum": 1}}}]
app_id_event_count_doc = events_collection.aggregate(pipeline)
app_id_event_count_dict  = {}

total_count = 0
for x in app_id_event_count_doc:
    app_id = x['_id'].strip()
    event_count = x['count']
    total_count += event_count
    app_id_event_count_dict[app_id] = event_count

app_id_event_count_dict =  OrderedDict(sorted(app_id_event_count_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True))
for app_id in app_id_event_count_dict:
    print(app_id," - ", app_id_app_name_dict[app_id], " - ", app_id_event_count_dict[app_id])

print("Total Event Count: ",total_count)

37f0228f2734c8010512d447a080fad3  -  stoffice  -  88117
bf57c906fa7d2bb66d07372e41585d96  -  excel  -  73844
cd0fbb7849b9b9d84c4f8e48e7301ecf  -  outlook  -  69798
a2b5ab0a0378213f941106a349e9304b  -  ifs.corp.lpl.com  -  55706
aa25d6af980471b133f7ad0f981aec16  -  vncecmxcp2prd.ncprod.lpl.com  -  23476
f6f4ab435f8857ee0528a815c0f6728c  -  lplecm.crm.dynamics.com  -  8526
0186dcc060cb68888b8a088ec9c9332e  -  msaccess  -  8195
9edc8b8fa912a3d810043b9f17ea7a8a  -  winword  -  217
Total Event Count:  327879


In [5]:
for app_id in app_id_event_count_dict:
    app_id_title_docs = events_collection.find({"application_id": app_id}, projection={'title':1, 'specifications.event_path':1})
    title_set = set({})
    event_path_set = set({})

    for x in app_id_title_docs:
        title_set.add(x['title'])
        event_path_set.add(x['specifications']['event_path'])

    f = open("../Schneider/"+app_id_app_name_dict[app_id].replace(":","_").replace("-","_")+"_title.txt",'w', encoding="utf-8")
    for title in title_set:
        f.write(title)
        f.write("\n")
    f.close()
    f = open("../Schneider/"+app_id_app_name_dict[app_id].replace(":","_").replace("-","_")+"_event_path.txt",'w', encoding="utf-8")
    for event_path in event_path_set:
        f.write(event_path)
        f.write("\n")
    f.close()

In [6]:
input_dir = "../Schneider/"
application_name = 'msaccess'.replace("-","_")


control_type_control_name_set_dict = {}
control_type_automation_id_set_dict = {}
control_type_automation_id_count_dict = {}

f = open(input_dir + application_name+"_event_path.txt",'r', encoding='utf-8')
lines = f.readlines()
f.close()

for line in lines:
    event_path = line.strip()
    if len(event_path) == 0:
        continue
    #if 'user' in event_path.lower():
    #    print(event_path)
    event_path_items = event_path.split("\/")
    for event_path_item in event_path_items:
        event_path_item_elements = event_path_item.split("\|")
        if len(event_path_item_elements) < 4:
            continue
        automation_id = event_path_item_elements[1][3:].strip()
        control_type = event_path_item_elements[2][2:].strip()
        control_name = event_path_item_elements[3][2:].strip()
        
        if control_type in control_type_control_name_set_dict:
            control_type_control_name_set = control_type_control_name_set_dict[control_type]
        else:
            control_type_control_name_set = set({})
        control_type_control_name_set.add(control_name)
        control_type_control_name_set_dict[control_type] = control_type_control_name_set
        
        if control_type in control_type_automation_id_set_dict:
            control_type_automation_id_set = control_type_automation_id_set_dict[control_type]
            
        else:
            control_type_automation_id_set = set({})
        control_type_automation_id_set.add(automation_id)
        control_type_automation_id_set_dict[control_type] = control_type_automation_id_set

        if control_type in control_type_automation_id_count_dict:
            automation_id_count_dict = control_type_automation_id_count_dict[control_type]
        else:
            automation_id_count_dict = {}
        count = 1
        if automation_id in automation_id_count_dict:
            count += automation_id_count_dict[automation_id]
        automation_id_count_dict[automation_id] = count
        control_type_automation_id_count_dict[control_type] = automation_id_count_dict
        
control_type_control_name_count_dict = {}
for control_type in control_type_control_name_set_dict:
    control_type_control_name_count_dict[control_type] = len(control_type_control_name_set_dict[control_type])
control_type_control_name_count_dict = OrderedDict(sorted(control_type_control_name_count_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True))

for control_type in control_type_automation_id_count_dict:
    automation_id_count = control_type_automation_id_count_dict[control_type]
    automation_id_count = OrderedDict(sorted(automation_id_count.items(), key=lambda kv:(kv[1], kv[0]), reverse=True))
    control_type_automation_id_count_dict[control_type] = automation_id_count
    
for control_type in control_type_control_name_count_dict:
    print(control_type, control_type_control_name_count_dict[control_type])

50033 122
50029 45
50000 41
50025 35
50032 26
50004 19
50020 12
50034 9
50003 9
50021 6
50011 6
50008 4
50030 2
50026 2
50018 2
50017 2
50014 2
50009 2
50007 2
50037 1
50036 1
50027 1
50006 1
50002 1


In [15]:
# print(control_type_control_name_set_dict)
for control_name in sorted(control_type_control_name_set_dict['50000']):
    # if "Realize Your Potential" in control_name:
        print(control_name)


#08260c55f970144dc6805a6ee34147599fc6b5b55ba956997e3f8737a4f7a6f7# right
#34e842da806432c225e7df004ab5a7a63725acf06bf387502dab64a568260248#
#3873ebc89146ed28bffb7315cb9d4dcc364e90ac566b1f437b7f31f0d3b4be28#
#48fb1f0348c3a0a8e3380f168c82a306fcf764085e8b828031f106a7c7191bf9#
#565339bc4d33d72817b583024112eb7f5cdf3e5eef0252d6ec1b9c9a94e12bb3#
#80d9e2df39466d8d7a372b59245908ad9598e07aa2d855cb397dba8a96c07daf# Macro
#85a39ab345d672ff8ca9b9c6876f3adcacf45ee7c1e2dbd2408fd338bd55e07e#
#caefb25ee1d75542b44d504117e9a55f27fbc300e00c86f118ad444e3d4e75a2#
#e502b1863ec58dde931460f3af865f24093c76785098fee472db5c6bc3c48b15# Macro
#f24ec859e4af152d5f19193d94978481d17ec21806b01db2c2335899609bdd8c# Macro
#f6bef75c7dcea2f0056e71ea9e0a4ef30f3db62fc502f5a3462ee0df858c43d9# Macro
'Main Navigation Form' Tab
'tbl_SecFlow_data' Tab
26
Add New Record
Cancel
Close
Close 'Main Navigation Form'
Close 'tbl#53bf20669201b019bc1536517e14d2e23444b1d170f1aca809d506dc696ac940#AccountValidationChecks#53bf20669201b019bc1536

In [17]:
automation_id_count_dict = control_type_automation_id_count_dict['50000']
for automation_id in automation_id_count_dict:
    print(automation_id, automation_id_count_dict[automation_id])

 223
2 33
6 20
1 8
UpButton 3
Minimize-Restore 3
7 2
{1AC14E77-02E7-4E5D-B744-2EB1AE5198B7}\SnippingTool.exe 1


In [9]:
def prune_hashes(sentence):
    regex = re.compile("#[A-Fa-f0-9]{64,}#", re.IGNORECASE)
    modified_sentence = sentence
    matches = re.findall(regex, modified_sentence)
    for match in matches:
        modified_sentence = modified_sentence.replace(match, '')

    return modified_sentence.strip()